# INT303 Assignment 1

Author: 

ID number: 

The URL I worked with is https://maoyan.com/board/4 

## Task 1

Imports library, which is also defined as `requirements.txt`

In [ ]:
import io
import math
import os
import random
import re
import time
import json                         # Serialization the Movie object to files
import bs4
import requests
from fontTools.ttLib import TTFont  # due the font issuse
from loguru import logger           # logger
from lxml import etree              # another way to parsing the elements in HTML

The class to identify the numbers in MaoYan website. This [package](https://pypi.org/project/maoyan-font-recognize/) is also writen by me, which is only contain the number identify part.

In [ ]:
class CharToNum():
    """identfy the number of maoyan websit.
    """

    def __init__(self):
        # base font
        self.font_base = TTFont("./iconfont0.woff")
        for i in range(0, 10):
            self.font_base["glyf"][self._base_number_to_code(i)].coordinates = sorted(
                list(self.font_base["glyf"][self._base_number_to_code(i)].coordinates))

    def _max_points_in_region(self, l, d):
        n = 0
        for dd in d:
            for ll in l:
                # Find the nearest point
                r = math.sqrt((dd[0]-ll[0])**2 + (dd[1]-ll[1])**2)
                if r <= 10:
                    n += 1
                    l.remove(ll)
                    break
        # Accuracy
        p = n / len(d)
        logger.debug(f"accuracy rate：{p*100}%")
        return p

    def _base_number_to_code(self, x):
        return {
            1: "uniE83C",
            2: "uniEA5E",
            3: "uniF6D1",
            4: "uniF882",
            5: "uniE39D",
            6: "uniF139",
            7: "uniF223",
            8: "uniF7A6",
            9: "uniEBF7",
            0: "uniF1C0"
        }[x]

    def set_new_fonts(self, font_new_path):
        self.font_new = TTFont(font_new_path)
        logger.debug(f"font_keys: {self.font_new['glyf'].keys()}")

    def char2num(self, new_code: str):
        new_code = "uni" + \
            new_code.encode("unicode_escape").decode()[-4:].upper()
        coordinates_new = sorted(
            list(self.font_new["glyf"][new_code].coordinates))
        one_hot = []
        for i in range(0, 10):
            coordinates_base = self.font_base["glyf"][self._base_number_to_code(
                i)].coordinates
            one_hot.append(self._max_points_in_region(
                coordinates_new, coordinates_base))

        return one_hot.index(max(one_hot))


The scrape class, which is used to get data from MaoYan website.

In [ ]:
class Pa():
    s = requests.Session()
    s.headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36",
        "Cookie": "__mta=220434983.1634193511952.1634450602151.1634450606007.51; uuid_n_v=v1; uuid=58BB89202CB911ECA55FB3CFA9262833BB3ECA70687D4F2B8EB54895C9C14E30; _csrf=8aa7cda1fbcecf89640e9400340425e65ab5feddcb835616893fbdacd2625110; _lxsdk_cuid=17c7d86155b1a-0934cb32140a51-1a2f1c08-1fa400-17c7d86155cc8; ci=55; featrues=[object Object]; Hm_lvt_703e94591e87be68cc8da0da7cbd0be2=1634193512,1634359559; _lxsdk=58BB89202CB911ECA55FB3CFA9262833BB3ECA70687D4F2B8EB54895C9C14E30; __mta=174488221.1634360855530.1634363365024.1634368138083.3; _lx_utm=utm_source=google&utm_medium=organic; Hm_lpvt_703e94591e87be68cc8da0da7cbd0be2=1634450606; _lxsdk_s=17c8cd8f564-087-17e-c6d||7"
    }

    def req(self, url):
        r = self.s.request("GET", url=url)
        r.encoding = "utf-8"
        return r.text

    def req_raw(self, url):
        r = self.s.request("GET", url=url)
        r.encoding = "utf-8"
        return r.content



The `Film` class is used to wrap the every file top 100 movie in MaoYan into object. Aims to OOP. 

In [ ]:
class Film:
    # init two modules to requests data and unrecognized number to int.  
    pa = Pa()
    char_to_num = CharToNum()

    def __init__(self, html: str):
        soup = bs4.BeautifulSoup(html, "html.parser")
        self.get_font(html)

        self.name_zh = soup.find_all("h1", class_="name")[0].contents[0]
        logger.info(f"{self.name_zh} is starting.")
        self.name_en = soup.find_all("div", class_="ename ellipsis")[0].contents[0]
        self.categorys = [category.contents[0].strip()
                          for category in soup.find_all("a", class_="text-link")]  # lambda?
        t = [i.strip() for i in soup.find_all(
            "li", class_="ellipsis")[1].contents[0].split("/")]
        self.area = t[0]
        self.duration = t[1]
        self.date = soup.find_all("li", class_="ellipsis")[2].contents[0]
        self.awards = [award.contents[1].contents[2].strip()
                       for award in soup("li", class_="award-item")]
        stuffs = [stuff.contents[0].strip()
                  for stuff in soup("a", class_="name")]
        self.director = stuffs[0]
        self.actors = stuffs[1:]
        spans = soup("span", class_="stonefont")
        rate_raw = spans[0].contents[0]
        money_unit = soup("span", class_="unit")[0].contents[0] if soup(
            "span", class_="unit") else 1
        self.rate = self._maoyan_formatter(rate_raw)
        rate_num_raw = spans[1].contents[0]
        self.rate_num = self._maoyan_formatter(rate_num_raw)
        if len(spans) == 3:
            money_raw = spans[2].contents[0]
            self.money = self._maoyan_formatter(money_raw, money_unit)
        else:
            self.money = -1

    def _maoyan_formatter(self, chars: bs4.element.NavigableString, money_unit=None):
        units_zh = {"万": 1e4, "亿": 1e8, "万美元": 63900.0}
        result = ""
        base = 1
        # to due with the money unit in the html
        if money_unit != None:
            base *= units_zh[money_unit]

        if chars[-1] in units_zh:
            base *= units_zh[chars[-1]]
            chars = chars[:-1]

        for char in chars:
            if char == ".":
                result += "."
                continue
            i = str(self.char_to_num.char2num(char))
            logger.debug("ans: ", i)
            result += i

        return float(result)*base

    def get_font(self, html):
        font_file = re.findall(
            r'vfile\.meituan\.net\/colorstone\/(\w+\.woff)', html)[0]
        font_url = 'https://vfile.meituan.net/colorstone/' + font_file
        logger.debug(f"font file is: {font_file}")
        font_bytes = self.pa.req_raw(font_url)
        font_fd = io.BytesIO(font_bytes)
        self.char_to_num.set_new_fonts(font_fd)



2 extra bytes in post.stringData array


The `MaoYan` class is used to get top 100 movies list on the ranking and get the movie detail respectively. 

1. The top 100 movies link will be stored into a file `top100_list_name` firstly. This is aims to increase the robustness to aviod interrupt when scrape the details of movies. 

2. Then it will scrape every top 100 movie detail page and store into a `{movie_name}.json` file (the movie_name is a variable). These json files prepare for the next analysis. 


Due to the MaoYan *anti-scrape strategy*, the following code contain a *re-scrape* strategy to recover from the intercept, which are `never_gona_give_you_up()` and `link_seeker()`.


In [ ]:
class MaoYan():
    path = "https://maoyan.com/board/4?offset="
    movie_prefix = "https://maoyan.com"
    pa = Pa()
    href_links = []
    moives = []
    
    def get_top_100_name(self):
        for offset in range(0, 100, 10):
            url = self.path + str(offset)
            txt = self.pa.req(url)
            # logger.info(f"contents in response: {c}")
            if "猫眼验证中心" in txt:
                # raise NeedForVerifyException("猫眼验证中心")
                logger.error("猫眼验证中心: need for verify code")
                break

            if "喵~好像哪里出错了唉.." in txt:
                logger.info("喵~好像哪里出错了唉..")
                break

            tree = etree.HTML(txt)
            for ctr in range(1, 11):
                el = tree.xpath(
                    f"//*[@id='app']/div/div/div[1]/dl/dd[{ctr}]/div/div/div[1]/p[1]/a")[0]
                href = el.attrib["title"]
                self.href_links.append(href)
                logger.info(f"{offset+ctr}, href: {href}")
            # small interval in every request
            self._wait()

        logger.info(f"All href links: {self.href_links}")
        with open("./top100_list_name", "w") as fd:
            fd.write(str(self.href_links))

    def get_top_100_link(self):
        for offset in range(0, 100, 10):
            url = self.path + str(offset)
            txt = self.pa.req(url)
            # logger.info(f"contents in response: {c}")
            if "猫眼验证中心" in txt:
                # raise NeedForVerifyException("猫眼验证中心")
                logger.error("猫眼验证中心: need for verify code")
                break

            if "喵~好像哪里出错了唉.." in txt:
                logger.info("喵~好像哪里出错了唉..")
                break

            tree = etree.HTML(txt)
            for ctr in range(1, 11):
                el = tree.xpath(
                    f"//*[@id='app']/div/div/div[1]/dl/dd[{ctr}]/div/div/div[1]/p[1]/a")[0]
                href = el.attrib["href"]
                self.href_links.append(href)
                logger.info(f"{offset+ctr}, href: {href}")
            # small interval in every request
            self._wait()

        logger.info(f"All href links: {self.href_links}")
        with open("./top100_list", "w") as fd:
            fd.write(str(self.href_links))

    def _wait(self):
        logger.debug("...waiting...")
        # choose a random time to stop. Maybe useful to pretend a human.
        time.sleep(random.uniform(10, 15))

    def get_detail(self):
        for index, link in enumerate(self.href_links):
            req_path = self.movie_prefix + link
            html = self.pa.req(req_path)
            f = Film(html)
            # self.moives.append(f)
            logger.success(f"<{index}>name: {f.name_zh} finish!")
            c = json.dumps(f.__dict__, ensure_ascii=False)
            with open(f"./movies/{f.name_zh}.json", "w") as fd:
                fd.write(c)

            self._wait()
        return True


def link_seeker(m, ori_href_link):
    movies_num = len(os.listdir("./movies"))
    m.href_links = ori_href_link[movies_num+1:]


def never_gona_give_you_up(m: MaoYan, ori_href_link: list):
    link_seeker(m, ori_href_link)
    while True:
        try:
            if m.get_detail():
                break
        except IndexError as e: # The IndexError is the most obvious feature of intercept by MapYan
            logger.error(e)
            link_seeker(m, ori_href_link)
            m._wait()


`main()` is determined the order sequence.

In [ ]:
def main():
    logger.level("INFO")
    os.makedirs("movies", exist_ok=True)
    m = MaoYan()
    if not os.path.exists("./top100_list"): 
        # get the top 100 movies link and store to a file.
        m.get_top_100_link()

    if not os.path.exists("./top100_list_name"): 
        m.get_top_100_name()

    # open the links file to get detail information respectively
    fd = open("./top100_list", "r")
    l = fd.read()
    ori_href_link = eval(l)
    m.href_links = ori_href_link
    fd.close()

    # get the movies without interrupted
    never_gona_give_you_up(m, ori_href_link)


Run! (The output is incomplete)

In [ ]:
main()

2021-11-03 21:48:52.599 | INFO     | __main__:get_top_100_link:28 - 1, href: /films/1200486
2021-11-03 21:48:52.600 | INFO     | __main__:get_top_100_link:28 - 2, href: /films/1297
2021-11-03 21:48:52.601 | INFO     | __main__:get_top_100_link:28 - 3, href: /films/1206605
2021-11-03 21:48:52.601 | INFO     | __main__:get_top_100_link:28 - 4, href: /films/1292
2021-11-03 21:48:52.602 | INFO     | __main__:get_top_100_link:28 - 5, href: /films/1211270
2021-11-03 21:48:52.603 | INFO     | __main__:get_top_100_link:28 - 6, href: /films/1203
2021-11-03 21:48:52.604 | INFO     | __main__:get_top_100_link:28 - 7, href: /films/1216365
2021-11-03 21:48:52.605 | INFO     | __main__:get_top_100_link:28 - 8, href: /films/1303
2021-11-03 21:48:52.605 | INFO     | __main__:get_top_100_link:28 - 9, href: /films/4055
2021-11-03 21:48:52.606 | INFO     | __main__:get_top_100_link:28 - 10, href: /films/46818
2021-11-03 21:48:52.607 | DEBUG    | __main__:_wait:37 - ...waiting...
2021-11-03 21:49:05.288 |

The next step is data precessing. Read all json file of movies into Pandas Dataframe. And generate the requested `.csv` file.

In [ ]:
import pandas as pd

In [ ]:
movie_files = os.listdir("./movies/")
movies = []

for movie_file in movie_files:
    with open(f"./movies/{movie_file}") as fd:
        movie = json.load(fd)
        movies.append(movie)


with open("./top100_list_name") as fd:
    ranking = eval(fd.read())


def add_ranking():
    d = {x["name_zh"]: x for x in movies}
    for index, name in enumerate(ranking):
        try:
            d[name]["ranking"] = index
        except KeyError as e:
            print(f"{e} did not apparent in the movies details.")

    return d


movies_dict = add_ranking()


df = pd.DataFrame(movies)

with open("MaoYanTop100.csv", "w") as fd:
    fd.write(df.to_csv())


## Task 2

Tell the story.

Import all needed packages.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import OrderedDict
from scipy.optimize import curve_fit
import seaborn as sns

In [ ]:
class AnalysisMaoYan():

    def __init__(self) -> None:
        self.df = pd.read_csv("MaoYanTop100.csv")
        self.df_clean = self.df.loc[self.df["money"] != -1]
        plt.rc('font', family='Sarasa Gothic CL')


    def actors_rank_all(self):
        actor_times = {}
        for actors in self.df["actors"]:
            for actor in eval(actors):
                if actor in actor_times:
                    actor_times[actor] += 1
                else:
                    actor_times[actor] = 1
        return OrderedDict(sorted(actor_times.items(), key=lambda i: i[1], reverse=True))

    def sub(self, m0: dict, m1: dict):
        """subtruct two dict, ignore m1 unique element. This will change m1.

        Args:
            m0 (dict)
            m1 (dict)
        """
        for i in m0:
            if i in m1:
                m0[i] -= m1[i]

    def actors_rank_major(self):

        actor_times = {}
        for actors in self.df["actors"]:
            for actor in eval(actors)[:5]:
                if actor in actor_times:
                    actor_times[actor] += 1
                else:
                    actor_times[actor] = 1

        self.sub(actor_times, self.directors_rank())

        return OrderedDict(sorted(actor_times.items(), key=lambda i: i[1], reverse=True))

    def directors_rank(self):
        directors_times = {}
        for directors in self.df["director"]:
            if directors in directors_times:
                directors_times[directors] += 1
            else:
                directors_times[directors] = 1
        return OrderedDict(sorted(directors_times.items(), key=lambda i: i[1], reverse=True))

    def category_rank(self):
        category_times = {}
        for categorys_movies in self.df["categorys"]:
            for category in eval(categorys_movies):
                if category in category_times:
                    category_times[category] += 1
                else:
                    category_times[category] = 1
        return sorted(category_times.items(), key=lambda i: i[1], reverse=True)

    def duration_distrubtion(self):
        durations = np.array(sorted([int(x[:-2])
                             for x in self.df["duration"]]))

        return durations

    def good_time(self):
        months = [x[5:7] for x in self.df["date"]]
        years = [x[:4] for x in self.df["date"]]
        return months, years

    def rate_num_distrubtion(self):
        rate_num = [x for x in self.df["rate_num"]]
        return rate_num

    def plot_types_pie(self):
        a = np.array(self.category_rank())
        x = list(a[:, 0])
        y = np.array(a[:, 1], dtype=int)
        colors = ['yellowgreen', 'red', 'gold', 'lightskyblue', 'white', 'lightcoral',
                  'blue', 'pink', 'darkgreen', 'yellow', 'grey', 'violet', 'magenta', 'cyan']
        porcent = 100.*y/y.sum()
        patches, texts = plt.pie(y, colors=colors, startangle=90, radius=1.2)
        labels = ['{0} - {1:1.2f} %'.format(i, j) for i, j in zip(x, porcent)]
        patches, labels, dummy = zip(*sorted(zip(patches, labels, y),
                                             key=lambda x: x[2],
                                             reverse=True))
        # plt.rc('font', family='Sarasa Gothic CL')
        plt.legend(patches, labels, bbox_to_anchor=(-0.1, 1))

        print(labels)
        plt.show()

    def plot_money_rate(self):
        vs = np.array([self.df_clean["money"], self.df_clean["rate"]],
                      dtype=(float, int)).T
        vs = np.sort(vs)
        # income
        x = vs[:, 1]
        # rate
        y = vs[:, 0]
        fig, ax = plt.subplots()

        params = np.polyfit(x, y, 2)
        xp = np.linspace(x.min(), x.max(), 20)
        yp = np.polyval(params, xp)
        plt.plot(xp, yp, 'k', alpha=0.8, linewidth=1)
        # plt.xticks(x, np.exp(x))
        ax.scatter(x, y, marker="o", alpha=0.5)

        ax.set(xlabel='Income (¥)', ylabel='Rating',
               title='The relationship between movies income against rating.')
        ax.grid()
        plt.show()

    def plot_money_rate_avg(self):
        vs = np.array([self.df_clean["rate"], self.df_clean["money"]]).T
        l = {}
        for x in vs:
            if x[0] in l:
                l[x[0]].append(x[1])
            else:
                l[x[0]] = [x[1]]

        for i in l:
            l[i] = np.average(l[i])

        print(l)
        vs = np.array(list(l.items()))
        vs = np.sort(vs)
        x = vs[:, 1]
        y = vs[:, 0]

        fig, ax = plt.subplots()

        params = np.polyfit(x, y, 2)
        xp = np.linspace(x.min(), x.max(), 20)
        yp = np.polyval(params, xp)
        plt.plot(xp, yp, 'k', alpha=0.8, linewidth=1)
        # plt.xticks(x, np.exp(x))
        ax.scatter(x, y, marker="o", alpha=0.5)
        sig = np.std(y - np.polyval(params, x))
        plt.fill_between(xp, yp - sig, yp + sig,
                         color='k', alpha=0.2)
        ax.set(xlabel='Income (¥)', ylabel='Rating',
               title='The relationship between movies income against rating.')
        ax.grid()
        plt.show()

    def income_director(self):
        # vs = dict self.df_clean["director"], self.df_clean["money"]
        vs = {}
        for index, i in self.df_clean.iterrows():
            if i["director"] in vs:
                vs[i["director"]].append(i["money"])
            else:
                vs[i["director"]] = [i["money"]]

        for i in vs:
            vs[i] = np.average(vs[i])

        vs = [[x[0], int(x[1])] for x in vs.items()]
        vs = sorted(vs, key=lambda x: x[1])
        vs = np.array(vs)
        x = vs[:, 0]
        y = vs[:, 1].astype(float)

        fig, ax = plt.subplots()
        # plt.xticks(x, np.exp(x))
        ax.scatter(x, y, marker="o", alpha=0.5)
        ax.set(xlabel='Director', ylabel='Income (¥)',
               title='movies income against director.')
        ax.grid()
        fig.autofmt_xdate()
        plt.show()
        print(vs)

    def plot_type_director(self):
        vs = {}
        for index, i in self.df.iterrows():
            if i["director"] in vs:
                vs[i["director"]].extend(eval(i["categorys"]))
            else:
                vs[i["director"]] = eval(i["categorys"])

        vs = [[x[0], x[1]] for x in vs.items()]
        vs = sorted(vs, key=lambda x: len(x[1]), reverse=True)[:9]
        print(vs)
        fig1, axs = plt.subplots(3, 3)

        for i, ax in zip(vs, axs.ravel()):
            name = i[0]
            types = i[1]
            d = {}
            for t in types:
                if t in d:
                    d[t] += 1
                else:
                    d[t] = 1
            v = np.array(list(d.items()))
            # Pie chart, where the slices will be ordered and plotted counter-clockwise:
            labels = v[:, 0]
            sizes = v[:, 1]
            # only "explode" the 2nd slice (i.e. 'Hogs')
            # explode = (0, 0.1, 0, 0)

            ax.pie(sizes, labels=labels, autopct='%1.1f%%',
                   shadow=True, startangle=90)
            ax.set_title(name)
            # Equal aspect ratio ensures that pie is drawn as a circle.
            ax.axis('equal')

        plt.show()


a = AnalysisMaoYan()
a.plot_type_director()
# print(a.df_clean)


def plot_his(x, y):
    num_bins = 10
    weight_bins = np.zeros(num_bins)
    interval = (x.max()-x.min())/num_bins
    for i in x:
        index = int((i - x.min()) // interval)
        weight_bins[index] += 1

    fig, ax = plt.subplots()

    # the histogram of the data
    n, bins, patches = ax.hist(x, num_bins, density=True)

    # add a 'best fit' line
    y = weight_bins
    # ax.plot(x, y, '--')
    ax.set_xlabel('rate number')
    ax.set_ylabel('number of rate number')
    # ax.set_title(r'Histogram of IQ: $\mu=100$, $\sigma=15$')

    # Tweak spacing to prevent clipping of ylabel
    fig.tight_layout()
    plt.show()
